In [1]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets
!pip install -q scann

In [2]:
REGION = "us-central1"
PROJECT_ID = !(gcloud config get-value project)
PROJECT_ID = PROJECT_ID[0]

DATASET = "movielens"
FOLDER = "match"
PIPELINE_JSON = f"{FOLDER}/{DATASET}_kfp_pipeline.json"

ARTIFACT_STORE = f"gs://kfp-{DATASET}-artifact-store-{PROJECT_ID}"
MODEL_DIR = f"{ARTIFACT_STORE}/model"
PIPELINE_ROOT = f"{ARTIFACT_STORE}/pipeline"
DATA_ROOT = f"{ARTIFACT_STORE}/data"
JOB_DIR_ROOT = f"{ARTIFACT_STORE}/jobs"
TRAINING_FILE_PATH = f"{DATA_ROOT}/training/dataset.csv"
TRAINING_CLEAN_FILE_PATH = f"{DATA_ROOT}/training_clean/dataset.csv"
VALIDATION_FILE_PATH = f"{DATA_ROOT}/validation/dataset.csv"
VALIDATION_CLEAN_FILE_PATH = f"{DATA_ROOT}/validation_clean/dataset.csv"

In [3]:
IMAGE_NAME = "trainer_image_movielens"
TAG = "latest"
TRAINING_CONTAINER_IMAGE_URI = f"gcr.io/{PROJECT_ID}/{IMAGE_NAME}:{TAG}"
SERVING_CONTAINER_IMAGE_URI = (
    "us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-3:latest"
)

In [4]:
%env DATASET={DATASET}
%env PIPELINE_ROOT={PIPELINE_ROOT}
%env PROJECT_ID={PROJECT_ID}
%env REGION={REGION}
%env JOB_DIR_ROOT={JOB_DIR_ROOT}
%env TRAINING_FILE_PATH={TRAINING_FILE_PATH}
%env VALIDATION_FILE_PATH={VALIDATION_FILE_PATH}
%env SERVING_CONTAINER_IMAGE_URI={SERVING_CONTAINER_IMAGE_URI}
%env TRAINING_CONTAINER_IMAGE_URI={TRAINING_CONTAINER_IMAGE_URI}

# Set `PATH` to include the directory containing KFP CLI
PATH = %env PATH
%env PATH=/home/jupyter/.local/bin:{PATH}

env: DATASET=movielens
env: PIPELINE_ROOT=gs://kfp-movielens-artifact-store-qwiklabs-gcp-04-853e5675f5e8/pipeline
env: PROJECT_ID=qwiklabs-gcp-04-853e5675f5e8
env: REGION=us-central1
env: JOB_DIR_ROOT=gs://kfp-movielens-artifact-store-qwiklabs-gcp-04-853e5675f5e8/jobs
env: TRAINING_FILE_PATH=gs://kfp-movielens-artifact-store-qwiklabs-gcp-04-853e5675f5e8/data/training/dataset.csv
env: VALIDATION_FILE_PATH=gs://kfp-movielens-artifact-store-qwiklabs-gcp-04-853e5675f5e8/data/validation/dataset.csv
env: SERVING_CONTAINER_IMAGE_URI=us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-3:latest
env: TRAINING_CONTAINER_IMAGE_URI=gcr.io/qwiklabs-gcp-04-853e5675f5e8/trainer_image_movielens:latest
env: PATH=/home/jupyter/.local/bin:/usr/local/cuda/bin:/opt/conda/bin:/opt/conda/condabin:/usr/local/bin:/usr/bin:/bin:/usr/local/games:/usr/games


In [5]:
import os

os.makedirs(FOLDER, exist_ok=True)

In [6]:
!gsutil ls | grep ^{ARTIFACT_STORE}/$ || gsutil mb -l {REGION} {ARTIFACT_STORE}

gs://kfp-movielens-artifact-store-qwiklabs-gcp-04-853e5675f5e8/


## Preprocessing

In [8]:
import pandas as pd
import numpy as np

df_train = pd.read_csv(TRAINING_FILE_PATH)
df_val = pd.read_csv(VALIDATION_FILE_PATH)
print(df_train.shape, f"Training path: {TRAINING_FILE_PATH}\n")
print(df_val.shape, f"Validation path: {VALIDATION_FILE_PATH}")

movies_ids = df_train.movieId.unique().tolist() + df_val.movieId.unique().tolist()
users_ids =  df_train.userId.unique().tolist() + df_val.userId.unique().tolist()

dict_movies = dict(zip(movies_ids, range(len(movies_ids)) ))
dict_users = dict(zip(users_ids, range(len(users_ids)) ))

df_train["movieId"] = df_train["movieId"].map(dict_movies)
df_val["movieId"] = df_val["movieId"].map(dict_movies)

df_train["userId"] = df_train["userId"].map(dict_users)
df_val["userId"] = df_val["userId"].map(dict_users)

col = ["userId", "movieId", "rating"]
df_train[col] = df_train[col].astype(np.float32)
df_val[col] = df_val[col].astype(np.float32)

# save data
df_train.to_csv(TRAINING_CLEAN_FILE_PATH, index=False)
df_val.to_csv(VALIDATION_CLEAN_FILE_PATH, index=False)

(7996867, 4) Training path: gs://kfp-movielens-artifact-store-qwiklabs-gcp-04-853e5675f5e8/data/training/dataset.csv

(2002053, 4) Validation path: gs://kfp-movielens-artifact-store-qwiklabs-gcp-04-853e5675f5e8/data/validation/dataset.csv


## Model training

In [ ]:
import os
import pprint
import tempfile
import pandas as pd

from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow.keras import Model
from tensorflow.keras import optimizers as opt
from tensorflow.keras.layers import Embedding, multiply, concatenate, Flatten, Input, Dense

from sklearn.model_selection import train_test_split

df = pd.read_parquet("gs://qwiklabs-gcp-03-c2cbc3b6d290-data/ml-25m/merged/df-rating-movie.parquet")

df_train, df_val = train_test_split(df, random_state=42, test_size=0.2, stratify=df.rating)

movies_ids = list(set(list(df_train.movieId.unique()) + list(df_val.movieId.unique())))
users_ids = list(set(list(df_train.userId.unique()) + list(df_val.userId.unique())))

dict_movies = {}
index = 0
for ids in sorted(movies_ids):
    dict_movies[ids] = index
    index += 1

dict_users = {}
index = 0
for ids in sorted(users_ids):
    dict_users[ids] = index
    index += 1

df_train["movieId"] = df_train["movieId"].map(dict_movies)
df_val["movieId"] = df_val["movieId"].map(dict_movies)

df_train["userId"] = df_train["userId"].map(dict_users)
df_val["userId"] = df_val["userId"].map(dict_users)

for col in ["userId", "movieId", "rating"]:
    df_train[col] = df_train[col].astype(np.float32)
    df_val[col] = df_val[col].astype(np.float32)

num_unique_users=len(set(list(df_train.userId.unique()) + list(df_val.userId.unique())))
num_unique_movies=len(set(list(df_train.movieId.unique()) + list(df_val.movieId.unique())))

users_input = Input(shape=(1,), name="users_input")
users_embedding = Embedding(num_unique_users + 1, 50, name="users_embeddings")(users_input)
users_bias = Embedding(num_unique_users + 1, 1, name="users_bias")(users_input)

movies_input = Input(shape=(1,), name="movies_input")
movies_embedding = Embedding(num_unique_movies + 1, 50, name="movies_embedding")(movies_input)
movies_bias = Embedding(num_unique_movies + 1, 1, name="movies_bias")(movies_input)

dot_product_users_movies = multiply([users_embedding, movies_embedding])
input_terms = dot_product_users_movies + users_bias + movies_bias
input_terms = Flatten(name="fl_inputs")(input_terms)
output = Dense(1, activation="relu", name="output")(input_terms)
model = Model(inputs=[users_input, movies_input], outputs=output)

opt_adam = opt.Adam(lr = 0.005)
model.compile(optimizer=opt_adam, loss= ['mse'], metrics=['mean_absolute_error'])

model.fit(x=[df_train.userId, df_train.movieId], y=df_train.rating, batch_size=512, epochs=3, verbose=1, validation_data=([df_val.userId, df_val.movieId], df_val.rating))

OUTPUT_DIR = "gs://qwiklabs-gcp-03-c2cbc3b6d290/kfp_tf/model"
#shutil.rmtree(OUTPUT_DIR, ignore_errors=True)
#TIMESTAMP = datetime.datetime.now().strftime("%Y%m%d%H%M%S")

EXPORT_PATH = os.path.join(OUTPUT_DIR, "1")

tf.saved_model.save(model, EXPORT_PATH)

OSError: Forbidden: b/qwiklabs-gcp-03-c2cbc3b6d290-data/o
1076138843678-compute@developer.gserviceaccount.com does not have storage.objects.list access to the Google Cloud Storage bucket.

In [9]:
# load preprocess_data
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras import optimizers as opt
from tensorflow.keras.layers import Embedding, multiply, concatenate, Flatten, Input, Dense

df_train = pd.read_csv(TRAINING_CLEAN_FILE_PATH)
df_val = pd.read_csv(VALIDATION_CLEAN_FILE_PATH)

num_unique_users=len(set(list(df_train.userId.unique()) + list(df_val.userId.unique())))
num_unique_movies=len(set(list(df_train.movieId.unique()) + list(df_val.movieId.unique())))

users_input = Input(shape=(1,), name="users_input")
users_embedding = Embedding(num_unique_users + 1, 50, name="users_embeddings")(users_input)
users_bias = Embedding(num_unique_users + 1, 1, name="users_bias")(users_input)

movies_input = Input(shape=(1,), name="movies_input")
movies_embedding = Embedding(num_unique_movies + 1, 50, name="movies_embedding")(movies_input)
movies_bias = Embedding(num_unique_movies + 1, 1, name="movies_bias")(movies_input)

dot_product_users_movies = multiply([users_embedding, movies_embedding])
input_terms = dot_product_users_movies + users_bias + movies_bias
input_terms = Flatten(name="fl_inputs")(input_terms)
output = Dense(1, activation="relu", name="output")(input_terms)
model = Model(inputs=[users_input, movies_input], outputs=output)

opt_adam = opt.Adam(lr = 0.005)
model.compile(optimizer=opt_adam, loss= ['mse'], metrics=['mean_absolute_error'])

model.fit(x=[df_train.userId, df_train.movieId], 
          y=df_train.rating, batch_size=512, 
          epochs=3, verbose=1, 
          validation_data=([df_val.userId, df_val.movieId], df_val.rating))

# save model
tf.saved_model.save(model, MODEL_DIR)

2022-03-17 15:43:58.466681: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2022-03-17 15:43:58.466834: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-03-17 15:43:58.466869: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (tensorflow-2-6): /proc/driver/nvidia/version does not exist
2022-03-17 15:43:58.467212: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/opt/conda/l

Epoch 1/3


InvalidArgumentError: Graph execution error:

Detected at node 'model/users_bias/embedding_lookup' defined at (most recent call last):
    File "/opt/conda/lib/python3.7/runpy.py", line 193, in _run_module_as_main
      "__main__", mod_spec)
    File "/opt/conda/lib/python3.7/runpy.py", line 85, in _run_code
      exec(code, run_globals)
    File "/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "/opt/conda/lib/python3.7/site-packages/traitlets/config/application.py", line 846, in launch_instance
      app.start()
    File "/opt/conda/lib/python3.7/site-packages/ipykernel/kernelapp.py", line 677, in start
      self.io_loop.start()
    File "/opt/conda/lib/python3.7/site-packages/tornado/platform/asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "/opt/conda/lib/python3.7/asyncio/base_events.py", line 541, in run_forever
      self._run_once()
    File "/opt/conda/lib/python3.7/async

# Official example

In [ ]:
import tensorflow_recommenders as tfrs
import tensorflow_datasets as tfds

In [ ]:
import os
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

import tensorflow_recommenders as tfrs

# Ratings data.
ratings = tfds.load("movielens/100k-ratings", split="train")
# Features of all the available movies.
movies = tfds.load("movielens/100k-movies", split="train")

In [ ]:
for x in ratings.take(1).as_numpy_iterator():
    pprint.pprint(x)

In [ ]:
for x in movies.take(1).as_numpy_iterator():
    pprint.pprint(x)

In [7]:
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"],
})
movies = movies.map(lambda x: x["movie_title"])

tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

In [10]:
movie_titles = movies.batch(1_000)
user_ids = ratings.batch(1_000_000).map(lambda x: x["user_id"])

unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

unique_movie_titles[:10]

array([b"'Til There Was You (1997)", b'1-900 (1994)',
       b'101 Dalmatians (1996)', b'12 Angry Men (1957)', b'187 (1997)',
       b'2 Days in the Valley (1996)',
       b'20,000 Leagues Under the Sea (1954)',
       b'2001: A Space Odyssey (1968)',
       b'3 Ninjas: High Noon At Mega Mountain (1998)',
       b'39 Steps, The (1935)'], dtype=object)

## Model

In [11]:
embedding_dimension = 32

user_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_user_ids, mask_token=None),
  # We add an additional embedding to account for unknown tokens.
  tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
])
movie_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_movie_titles, mask_token=None),
  tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
])
# metric
metrics = tfrs.metrics.FactorizedTopK(
  candidates=movies.batch(128).map(movie_model)
)
# loss
task = tfrs.tasks.Retrieval(
  metrics=metrics
)

In [12]:
class MovielensModel(tfrs.Model):

    def __init__(self, user_model, movie_model):
        super().__init__()
        self.movie_model: tf.keras.Model = movie_model
        self.user_model: tf.keras.Model = user_model
        self.task: tf.keras.layers.Layer = task

    def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
        # We pick out the user features and pass them into the user model.
        user_embeddings = self.user_model(features["user_id"])
        # And pick out the movie features and pass them into the movie model,
        # getting embeddings back.
        positive_movie_embeddings = self.movie_model(features["movie_title"])

        # The task computes the loss and the metrics.
        return self.task(user_embeddings, positive_movie_embeddings)

In [13]:
model = MovielensModel(user_model, movie_model)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()
model.fit(cached_train, epochs=1)
model.evaluate(cached_test, return_dict=True)

5/5 [==============================] - 5s 504ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0014 - factorized_top_k/top_5_categorical_accuracy: 0.0142 - factorized_top_k/top_10_categorical_accuracy: 0.0294 - factorized_top_k/top_50_categorical_accuracy: 0.1506 - factorized_top_k/top_100_categorical_accuracy: 0.2666 - loss: 31729.5866 - regularization_loss: 0.0000e+00 - total_loss: 31729.5866


{'factorized_top_k/top_1_categorical_accuracy': 0.0013500000350177288,
 'factorized_top_k/top_5_categorical_accuracy': 0.0142000000923872,
 'factorized_top_k/top_10_categorical_accuracy': 0.029400000348687172,
 'factorized_top_k/top_50_categorical_accuracy': 0.15060000121593475,
 'factorized_top_k/top_100_categorical_accuracy': 0.26660001277923584,
 'loss': 28856.1484375,
 'regularization_loss': 0,
 'total_loss': 28856.1484375}

## Prediction

In [14]:
# BRUTE FORCE LAYER

# Create a model that takes in raw query features, and
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)

# recommends movies out of the entire movies dataset.
index.index_from_dataset(
  tf.data.Dataset.zip((movies.batch(100), movies.batch(100).map(model.movie_model)))
)

# Get recommendations.
_, titles = index(tf.constant(["42"]))
print(f"Recommendations for user 42: {titles[0, :3]}")

Recommendations for user 42: [b'Nutty Professor, The (1996)' b'Twister (1996)'
 b'While You Were Sleeping (1995)']
